In [1]:
import torchvision
import torch
import argparse
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
from resnet import *
import torch.optim as optim
from utils import progress_bar
import time

In [2]:
parser=argparse.ArgumentParser()
parser.add_argument('--use-cuda',action='store_true',help='use cuda')
args=parser.parse_args()

usage: ipykernel_launcher.py [-h] [--use-cuda]
ipykernel_launcher.py: error: unrecognized arguments: -f /Users/yuesun/Library/Jupyter/runtime/kernel-58ad2c68-e74c-4519-9775-a7c0ecb02c5c.json


SystemExit: 2

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3405: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torchdevice=torch.device(device)

In [5]:
transform=transforms.Compose([
    transforms.RandomCrop(32,padding=4),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    
])

In [6]:
train_data=torchvision.datasets.CIFAR10(root='./data',train=True,download=True,transform =transform)
test_data=torchvision.datasets.CIFAR10(root='./data',train=False,download=True,transform =transform)

Files already downloaded and verified
Files already downloaded and verified


In [12]:

kwargs = {'pin_memory': True} if device=='cuda' else {}
trainloader = torch.utils.data.DataLoader(train_data, batch_size=128,
                                          shuffle=True,num_workers=0,**kwargs)
testloader = torch.utils.data.DataLoader(test_data, batch_size=100,
                                          shuffle=True,num_workers=0,**kwargs)

net = ResNet18()
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1,
                      momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

def train_model(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    dataloading_time=0
    training_time=0
    dataloading_start=time.perf_counter()
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        dataloading_end=time.perf_counter()
        dataloading_time+=(dataloading_end-dataloading_start)
        
        training_start=time.perf_counter()
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        training_end=time.perf_counter()
        training_time+=(training_end-training_start)
        
        
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        progress_bar(batch_idx, len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                     % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))

        dataloading_start=time.perf_counter()
    total_time=dataloading_time+training_time
    print("data loading time for this epoch: ",dataloading_time,"training time for this epoch: ",training_time,"total time for this epoch: ",total_time)
    return dataloading_time,training_time,total_time
    
    
def test_model(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            progress_bar(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                         % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))


best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch
dataloadtime_time_all_epoch=0
training_time_all_epoch=0
total_time_all_epoch=0
for epoch in range(start_epoch, start_epoch+5):
    dataloading_time,training_time,total_time=train_model(epoch)
    dataloadtime_time_all_epoch+=dataloading_time
    training_time_all_epoch+=training_time
    total_time_all_epoch+=total_time
    #test_model(epoch)
    scheduler.step()
    
print("data loading time for all epochs: ",dataloadtime_time_all_epoch/5,"training time for all epochs: ",training_time_all_epoch/5,"total time for all epochs: ",total_time_all_epoch/5)





Epoch: 0
 [================================================================>]  Step: 1s630ms | Tot: 18m45s | Loss: 1.954 | Acc: 30.160% (15080/5000 391/391 1 
data loading time for this epoch:  23.152794074002486 training time for this epoch:  1104.110671724994 total time for this epoch:  1127.2634657989965

Epoch: 1
 [================================================================>]  Step: 1s895ms | Tot: 17m26s | Loss: 1.458 | Acc: 46.544% (23272/5000 391/391   
data loading time for this epoch:  21.336233970992907 training time for this epoch:  1029.2299149980026 total time for this epoch:  1050.5661489689955

Epoch: 2
 [================================================================>]  Step: 1s664ms | Tot: 18m21s | Loss: 1.182 | Acc: 57.512% (28756/5000 391/391 1 
data loading time for this epoch:  22.163357160998203 training time for this epoch:  1082.496750458984 total time for this epoch:  1104.6601076199822

Epoch: 3
 [=========================================================